In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [2]:
data_features = pd.DataFrame()
for i in range(10):
    filename = "./safety/features/part-0000"+str(i)+"-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv"
    temp_data_features = pd.read_csv(filename, header=0)
    data_features = data_features.append(temp_data_features)

In [12]:
filename = "./safety/labels/part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv"
data_labels = pd.read_csv(filename,header=0)

In [3]:
data_features.shape

(16135561, 11)

In [5]:
data_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16135561 entries, 0 to 1613561
Data columns (total 11 columns):
bookingID         int64
Accuracy          float64
Bearing           float64
acceleration_x    float64
acceleration_y    float64
acceleration_z    float64
gyro_x            float64
gyro_y            float64
gyro_z            float64
second            float64
Speed             float64
dtypes: float64(10), int64(1)
memory usage: 1.4 GB


#Check for Duplicates

In [59]:
def features_remove_duplicates(df):
    df.drop_duplicates()
    return df

def labels_remove_duplicates(df):
    df.drop_duplicates("bookingID", keep="last")

##Check for Null/NA

In [6]:
data_features.describe()

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed
count,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07,1.613556e+07
mean,8.184819e+11,1.161035e+01,1.689684e+02,6.914529e-02,4.468976e+00,8.942974e-01,-1.710847e-03,2.800733e-04,-2.569688e-04,3.803484e+03,9.006543e+00
std,4.952476e+11,8.691672e+01,1.072966e+02,1.424161e+00,8.130664e+00,3.251549e+00,1.445619e-01,3.400630e-01,1.480824e-01,1.436686e+06,8.106415e+00
min,0.000000e+00,7.500000e-01,0.000000e+00,-7.841969e+01,-7.299412e+01,-7.844842e+01,-4.845575e+01,-7.488861e+01,-5.355445e+01,0.000000e+00,-2.000000e+00
25%,3.779571e+11,3.900000e+00,7.800000e+01,-5.081238e-01,-2.022476e+00,-9.316101e-01,-2.678888e-02,-2.993851e-02,-1.876500e-02,2.410000e+02,1.020000e+00
50%,8.074539e+11,4.250000e+00,1.688700e+02,6.137085e-02,9.081485e+00,7.757450e-01,-6.432411e-04,2.645046e-04,-3.542004e-05,5.200000e+02,7.530000e+00
75%,1.254130e+12,8.000000e+00,2.629840e+02,6.350620e-01,9.709778e+00,2.750938e+00,2.330639e-02,3.143085e-02,1.823425e-02,8.630000e+02,1.548000e+01
max,1.709397e+12,6.070101e+03,3.599995e+02,6.687346e+01,7.505589e+01,7.805576e+01,3.983975e+01,8.031496e+01,6.630078e+01,1.495797e+09,1.480186e+02


# Anomalies in data
* GPS Accuracy Max = 6070m
* Max Duration = 1.4e9 seconds
* Max Speed = 148 m/s = 532 kmh


In [88]:
def clean_features(df):
    #rename to more uniform convention
    df.columns = ['bookingID', 'accuracy', 'bearing', 
                    'acceleration_x', 'acceleration_y', 'acceleration_z',
                    'gyro_x', 'gyro_y', 'gyro_z',
                    'second','speed']
    #remove NAs
    df = df.dropna()
    df = df.drop_duplicates()
    df = df.loc[(df["accuracy"]<20) &
                (df["second"]<1e6) &
                (df["speed"]>0) &
                (df["speed"]<60)]
    return df

def clean_labels(df):
    df = df.dropna()
    df = df.drop_duplicates("bookingID", keep="last") #some booking IDs have multiple labels. set them to be dangerous
    return df

def process_data(features, labels):
    temp_features = clean_features(features)
    temp_labels = clean_labels(labels)
    data_combined = data = pd.merge(temp_features, temp_labels, how="inner",on="bookingID")
    data_combined.sort_values(["bookingID","second"], inplace=True)
    return data_combined

data_combined = process_data(data_features, data_labels)

In [87]:
data_combined.to_csv(r'./data_combined.csv')